In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hardikinani10","key":"2c3546aa1501e11b2ad85a5d625a09f2"}'}

In [2]:
!pip install -q kaggle

In [4]:
!mkdir -p ~/.kaggle

In [5]:
!cp kaggle.json ~/.kaggle/

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!mkdir dogbreed

In [8]:
%cd dogbreed

/content/dogbreed


In [9]:
!kaggle datasets download -d catherinehorng/dogbreedidfromcomp

100% 688M/691M [00:04<00:00, 186MB/s]
100% 691M/691M [00:04<00:00, 151MB/s]


In [ ]:
!unzip dogbreedidfromcomp.zip

In [15]:
%cd ..

/content


In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize

In [17]:
all_labels=pd.read_csv("/content/dogbreed/labels.csv")

In [18]:
all_labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [20]:
all_labels['breed'].value_counts()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
golden_retriever         67
brabancon_griffon        67
komondor                 67
eskimo_dog               66
briard                   66
Name: breed, Length: 120, dtype: int64

In [23]:
class_name=['scottish_deerhound','maltese_dog','bernese_mountain_dog']
labels=all_labels[(all_labels['breed'].isin(class_name))]
labels = labels.reset_index()

In [27]:
X_data=np.zeros((len(labels),224,224,3),dtype='float32')
Y_data=label_binarize(labels['breed'],classes=class_name)

In [30]:
print(X_data.shape)
print(Y_data.shape)

(357, 224, 224, 3)
(357, 3)


In [31]:
from keras.utils import img_to_array
import cv2
import tensorflow as tf

In [35]:
for i in range(len(labels)):
  img=cv2.imread("/content/dogbreed/train/%s.jpg" %labels['id'][i])
  img=cv2.resize(img,(224,224))
  img = img_to_array(img)
  x=np.expand_dims(img.copy(),axis=0)
  X_data[i] = x/255.0

In [37]:
xtrain,xtest,ytrain,ytest = train_test_split(X_data,Y_data,random_state=0)
xnew_train,xvalid,ynew_train,yvalid=train_test_split(xtrain,ytrain)

In [41]:
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,Dense,MaxPool2D
from keras.optimizers import Adam

In [45]:
from keras.utils.data_utils import Sequence
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(224,224,3),activation='relu'))
model.add(Conv2D(32,(3,3), activation='relu',kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(3,3)))
model.add(Conv2D(32,(3,3), activation='relu',kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(3,3)))
model.add(Conv2D(32,(3,3), activation='relu',kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(3,3)))
model.add(Flatten())
model.add(Dense(20,activation='relu'))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 conv2d_7 (Conv2D)           (None, 220, 220, 32)      9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 71, 71, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 21, 21, 32)       

In [46]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(0.001),metrics=['accuracy'])

In [47]:
hist = model.fit(xnew_train,ynew_train,epochs=2,batch_size=32,validation_data=(xvalid,yvalid))

Epoch 1/2
7/7 [==============================] - 38s 5s/step - loss: 2.0084 - accuracy: 0.3250 - val_loss: 1.9047 - val_accuracy: 0.3881
Epoch 2/2
7/7 [==============================] - 38s 6s/step - loss: 1.8650 - accuracy: 0.3850 - val_loss: 1.7643 - val_accuracy: 0.4478


In [48]:
import numpy as np
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [49]:
# Get the predicted labels
y_pred = model.predict(xtest)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get the true labels
y_true_classes = np.argmax(ytest, axis=1)

3/3 [==============================] - 6s 2s/step


In [51]:
# Generate the classification report
report = classification_report(y_true_classes, y_pred_classes)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       0.31      0.96      0.47        26
           1       0.00      0.00      0.00        30
           2       0.78      0.21      0.33        34

    accuracy                           0.36        90
   macro avg       0.36      0.39      0.26        90
weighted avg       0.38      0.36      0.26        90



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
model.save('dogbreed.h5')